# Day 19
## Part 1
~~I'm not bothering with this one.~~ Ok, the single empty day is nagging at me. Let's take this one slowly and not spend entire days debugging it.

The idea is for each scanner to calculate the absolute differences on each axis between each detected pair of beacons, forming a matrix of three differences for each pair. Represent this as a graph. Then, for each pair of scanners, calculate the overlapping distances. The pair of scanners with the highest number of mutual distances can combine their beacon maps into one by manipulating the axes of one to match the other. Repeat until their is only one beacon map left.

Parse the scanners as lists of three-coordinate tuples.

In [1]:
import parse
import itertools
import networkx as nx
from collections import Counter
from itertools import count

def parse_data(s):
    scanners = []
    for block in s.strip().split('\n\n'):
        scanner = []
        for line in block.splitlines():
            if r := parse.parse('{x:d},{y:d},{z:d}', line):
                scanner.append((r['x'], r['y'], r['z']))
        scanners.append(scanner)
    return scanners

test_data = parse_data(open('test_input', 'r').read())
data = parse_data(open('input', 'r').read())

Record the distance between each scanner's observations as a set of absolute differences in each coordinate direction, which will allow comparison with other scanners' coordinate systems.

In [2]:
def beacon_map(scanner):
    distances = nx.Graph()
    for (i, (x_i, y_i, z_i)), (j, (x_j, y_j, z_j)) in itertools.combinations(enumerate(scanner), 2):
        distance = frozenset([abs(d) for d in [x_i - x_j, y_i - y_j, z_i - z_j]])
        distances.add_edge(i, j, distance=distance)
        distances.nodes[i]['coordinate'] = (x_i, y_i, z_i)
        distances.nodes[j]['coordinate'] = (x_j, y_j, z_j)
    return distances

test_beacon_distances = [beacon_map(x) for x in test_data]
beacon_distances = [beacon_map(x) for x in data]

Put each of the distances into a set for each scanner.

In [3]:
def distances_in_graphs(beacon_maps):
    result = []
    for b in beacon_maps:
        result.append({b.edges[i, j]['distance'] for i, j in b.edges})
    return result

test_set_distances = distances_in_graphs(test_beacon_distances)
set_distances = distances_in_graphs(beacon_distances)

Count the mutual distances between them.

In [4]:
def count_mutual_distances(set_distances):
    result = []
    for (i, sdi), (j, sdj) in itertools.combinations(enumerate(set_distances), 2):
        result.append((i, j, len(sdi & sdj)))
    return(result)

test_mutual_distances = count_mutual_distances(test_set_distances)
mutual_distances = count_mutual_distances(set_distances)

Find the ids of the scanners with the highest number of mutual distances.

In [5]:
def most_overlapping_scanner_pair(mutual_distances):
    i, j, _ = max(mutual_distances, key=lambda x: x[2])
    return i, j
    
test_most_overlapping = most_overlapping_scanner_pair(test_mutual_distances)
test_most_overlapping

(0, 1)

For these scanners find the distances from each beacon to other beacons. Then compare between the two sets to match nodes.

In [6]:
def common_distances(beacons_1, beacons_2):
    return ({beacons_1.edges[edge]['distance'] for edge in beacons_1.edges}
            & {beacons_2.edges[edge]['distance'] for edge in beacons_2.edges})

def node_distances(beacons, distances):
    return {
        node: {
            beacons[node][nbr]['distance'] 
            for nbr in beacons[node]
            if beacons[node][nbr]['distance'] in distances
        } 
        for node in beacons
    }

test_common_distances = common_distances(test_beacon_distances[0], test_beacon_distances[1])
test_node_distances_0 = node_distances(test_beacon_distances[0], test_common_distances)
test_node_distances_1 = node_distances(test_beacon_distances[1], test_common_distances)

The node distances can then be matched to determine which beacons are the same within the different coordinate systems.

In [7]:
def match_nodes(node_distances_1, node_distances_2):
    possible_matches = []
    for node_1, node_2 in itertools.product(node_distances_1, node_distances_2):
        overlap = node_distances_1[node_1] & node_distances_2[node_2]
        if len(overlap) > 1: 
            possible_matches.append(((node_1, node_2), len(overlap)))
    return sorted(possible_matches, key=lambda x: x[1], reverse=True)

In [8]:
test_matched_nodes = match_nodes(test_node_distances_0, test_node_distances_1)
test_matched_nodes

[((0, 3), 11),
 ((1, 8), 11),
 ((3, 12), 11),
 ((4, 1), 11),
 ((5, 24), 11),
 ((6, 18), 11),
 ((7, 10), 11),
 ((9, 0), 11),
 ((12, 2), 11),
 ((14, 5), 11),
 ((19, 15), 11),
 ((24, 19), 11)]

Find a pair of nodes with unambiguous manhattan distances, i.e. the absolute distance on each axis is unique. 

In [9]:
def find_unambiguous_matches(beacons, matched_nodes):
    first = matched_nodes[0][0][0]
    for i in range(1, len(matched_nodes)):
        second = matched_nodes[i][0][0]
        # set will have three elements if all distinct
        if len(beacons[first][second]['distance']) == 3:
            return (0, i)
        
find_unambiguous_matches(test_beacon_distances[0], test_matched_nodes)

(0, 1)

The tricky bit. For matching pairs of manhattan distances align each axis and indicate if the sign of the difference differs. Then each coordinate for the second set of beaconds can be transformed to the coordinate system of the first.

In [10]:
def manhattan_by_axis(coord_1, coord_2):
    return tuple(coord_1[i] - coord_2[i] for i in range(3))

def calc_transform(coord_11, coord_12, coord_21, coord_22):
    diff_1 = manhattan_by_axis(coord_11, coord_12)
    diff_2 = manhattan_by_axis(coord_21, coord_22)
    axis_match = {}
    sign_match = {}
    for i, j in itertools.product(range(3), repeat=2):
        if abs(diff_1[i]) == abs(diff_2[j]):
            axis_match[j] = i
            if diff_1[i] == diff_2[j]:
                sign_match[j] = 1
            else:
                sign_match[j] = -1
    offset = [(coord_11[axis_match[j]] - coord_21[j]) for j in range(3)]
    origin_1 = tuple(coord_11[axis_match[j]] for j in range(3))
    origin_2 = coord_21
    return axis_match, sign_match, origin_1, origin_2

def transform(coord, axis_match, sign_match, origin_1, origin_2):
    transformed = {axis_match[i]: (coord[i] - origin_2[i]) * sign_match[i] + origin_1[i] for i in range(3)}
    return tuple(transformed[i] for i in range(3))

In [11]:
ct = calc_transform((404, -588, -901), (528, -643, 409), (-336, 658, 858), (-460, 603, -452))
ct

({0: 0, 1: 1, 2: 2}, {0: -1, 1: 1, 2: -1}, (404, -588, -901), (-336, 658, 858))

In [12]:
transform((-336, 658, 858), *ct)

(404, -588, -901)

In [13]:
transform((-460, 603, -452), *ct)

(528, -643, 409)

In [14]:
transform((515,917,-361), *ct)

(-447, -329, 318)

Putting it all together, repeatedly transform pairs of coordinate systems of scanner readings into one until only one is left.

In [15]:
def create_system(data):
    beacon_distances = [beacon_map(x) for x in data]
    while len(beacon_distances) > 1:
        set_distances = distances_in_graphs(beacon_distances)
        mutual_distances = count_mutual_distances(set_distances)
        i, j = most_overlapping_scanner_pair(mutual_distances)
        cds = common_distances(beacon_distances[i], beacon_distances[j])
        node_distances_i = node_distances(beacon_distances[i], cds)
        node_distances_j = node_distances(beacon_distances[j], cds)
        
        matched_nodes = match_nodes(node_distances_i, node_distances_j)
        first, second = find_unambiguous_matches(beacon_distances[i], matched_nodes)
        coord_i1 = beacon_distances[i].nodes[matched_nodes[first][0][0]]['coordinate']
        coord_i2 = beacon_distances[i].nodes[matched_nodes[second][0][0]]['coordinate']
        coord_j1 = beacon_distances[j].nodes[matched_nodes[first][0][1]]['coordinate']
        coord_j2 = beacon_distances[j].nodes[matched_nodes[second][0][1]]['coordinate']
        
        ct = calc_transform(coord_i1, coord_i2, coord_j1, coord_j2)
        # Should really have used the coordinate as the node's label, I've left this for 
        # so long I can't remember why I didn't
        existing_nodes = list(beacon_distances[i].nodes)
        existing_coords = {
            beacon_distances[i].nodes[n]['coordinate']
            for n in existing_nodes
        }
        new_node_indices = count(max(existing_nodes) + 1)
        for new_node_index, n in zip(new_node_indices, beacon_distances[j].nodes):
            coord = beacon_distances[j].nodes[n]['coordinate']
            transformed_coord = transform(coord, *ct)
            
            if transformed_coord not in existing_coords:
                current_nodes = list(beacon_distances[i].nodes)
                for n_i in current_nodes:
                    coord_i = beacon_distances[i].nodes[n_i]['coordinate']
                    distance = frozenset([abs(coord_i[x] - transformed_coord[x]) for x in range(3)])
                    beacon_distances[i].add_edge(n_i, new_node_index, distance=distance)
                    beacon_distances[i].nodes[new_node_index]['coordinate'] = transformed_coord
        del beacon_distances[j]
        
    return beacon_distances[0]

In [16]:
test_system = create_system(test_data)

In [17]:
len(test_system.nodes)

79

In [18]:
system = create_system(data)
len(system.nodes)

306

## Part 2

Change some of the previous code so that it always chooses the first scanner and transforms other coordinates into that system. The transformation of (0, 0, 0) for each set of coordinate system is then the positions of the other scanners relative to the first.

In [19]:
def most_overlapping_scanner_pair(mutual_distances):
    i, j, _ = max(((i, j, d) for i, j, d in mutual_distances if i == 0), key=lambda x: x[2])
    return i, j

def find_scanners(data):
    beacon_distances = [beacon_map(x) for x in data]
    scanners = [(0, 0, 0)]
    while len(beacon_distances) > 1:
        set_distances = distances_in_graphs(beacon_distances)
        mutual_distances = count_mutual_distances(set_distances)
        i, j = most_overlapping_scanner_pair(mutual_distances)
        cds = common_distances(beacon_distances[i], beacon_distances[j])
        node_distances_i = node_distances(beacon_distances[i], cds)
        node_distances_j = node_distances(beacon_distances[j], cds)
        
        matched_nodes = match_nodes(node_distances_i, node_distances_j)
        first, second = find_unambiguous_matches(beacon_distances[i], matched_nodes)
        coord_i1 = beacon_distances[i].nodes[matched_nodes[first][0][0]]['coordinate']
        coord_i2 = beacon_distances[i].nodes[matched_nodes[second][0][0]]['coordinate']
        coord_j1 = beacon_distances[j].nodes[matched_nodes[first][0][1]]['coordinate']
        coord_j2 = beacon_distances[j].nodes[matched_nodes[second][0][1]]['coordinate']
        
        ct = calc_transform(coord_i1, coord_i2, coord_j1, coord_j2)
        # Should really have used the coordinate as the node's label, I've left this for 
        # so long I can't remember why I didn't
        existing_nodes = list(beacon_distances[i].nodes)
        existing_coords = {
            beacon_distances[i].nodes[n]['coordinate']
            for n in existing_nodes
        }
        if i == 0:
            scanners.append(transform((0, 0, 0), *ct))
        new_node_indices = count(max(existing_nodes) + 1)
        for new_node_index, n in zip(new_node_indices, beacon_distances[j].nodes):
            coord = beacon_distances[j].nodes[n]['coordinate']
            transformed_coord = transform(coord, *ct)
            
            if transformed_coord not in existing_coords:
                current_nodes = list(beacon_distances[i].nodes)
                for n_i in current_nodes:
                    coord_i = beacon_distances[i].nodes[n_i]['coordinate']
                    distance = frozenset([abs(coord_i[x] - transformed_coord[x]) for x in range(3)])
                    beacon_distances[i].add_edge(n_i, new_node_index, distance=distance)
                    beacon_distances[i].nodes[new_node_index]['coordinate'] = transformed_coord
        del beacon_distances[j]
        
    return scanners

In [20]:
test_scanners = find_scanners(test_data)

In [21]:
test_scanners

[(0, 0, 0),
 (68, -1246, -43),
 (-92, -2380, -20),
 (-20, -1133, 1061),
 (1105, -1205, 1229)]

In [22]:
def manhattan(c1, c2):
    return sum(abs(x) for x in manhattan_by_axis(c1, c2))

def part_2(scanners):
    return max(manhattan(x, y) for x, y in itertools.combinations(scanners, 2))

In [23]:
part_2(test_scanners)

3621

In [24]:
scanners = find_scanners(data)
part_2(scanners)

9764